In [1]:
from natsort import natsorted
import pandas as pd
from glob import glob
import os

In [ ]:
!unzip -qq /content/drive/MyDrive/4-1_Capstone/CapstoneDataset/images_concatenate.zip -d /content/capstone_images
capstone_images = natsorted(glob(os.path.join("/content/capstone_images", '*')))

SAMPLES = 500
os.system("unzip -qq /content/drive/MyDrive/4-1_Capstone/AI_HubDataset/images" + str(SAMPLES) + ".zip -d /content/aihub_images")
aihub_images = natsorted(glob(os.path.join("/content/aihub_images/images" + str(SAMPLES), '*')))


images = capstone_images + aihub_images

In [3]:
%cd /content
!git clone https://github.com/IyatomiLab/LeafGAN

%cd /content/LeafGAN

!pip install -r requirements.txt -q
!pip install dominate -q

/content
fatal: destination path 'LeafGAN' already exists and is not an empty directory.
/content/LeafGAN
ERROR: Could not find a version that satisfies the requirement skimage>=0.16.1 (from versions: 0.0)
ERROR: No matching distribution found for skimage>=0.16.1


In [4]:
    # discriminator lr *= 0.1
!cp /content/drive/MyDrive/4-1_Capstone/temp/leaf_gan_model.py /content/LeafGAN/models/leaf_gan_model.py
    # 필요없는 이미지 저장하지 않음(real, fake만 저장)
!cp /content/drive/MyDrive/4-1_Capstone/temp/visualizer.py /content/LeafGAN/util/visualizer.py

import sys
sys.path.append("/content/drive/MyDrive/4-1_Capstone")

from utils import get_disease_healthy_map, get_crop_mask
disease_healthy_map = get_disease_healthy_map()

crop_mask = get_crop_mask()

In [5]:
# from torch.utils.data import Dataset
# import torchvision.transforms as T
# !pip install timm -q
# import torch
# import torch.nn as nn
# import timm
# from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD

# class MyDataset(Dataset):

#     def __init__(self, transform=None, gen_images=None):        
#         self.transform = transform
#         self.img_dirs = gen_images

#     def __len__(self):
#         return len(self.img_dirs)

#     def __getitem__(self, idx):        
#         image = Image.open(self.img_dirs[idx])
#         image = image.convert('RGB')
#         image = self.transform(image)
#         return image

# tf = T.Compose([
#     T.Resize(224, interpolation=3),
#     T.ToTensor(),
#     T.Normalize(IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD),
# ])

In [6]:
# N_CLASSES = 72

# # model = timm.create_model(model_name = 'efficientnet_b0', pretrained=False)
# # model.classifier = nn.Linear(1280, N_CLASSES)
# # model.load_state_dict(torch.load("/content/drive/MyDrive/4-1_Capstone/models/E_10.pth"))

# from PIL import Image
# import requests
# import matplotlib.pyplot as plt
# %config InlineBackend.figure_format = 'retina'

# import torch
# import timm
# import torchvision
# import torchvision.transforms as T
# from timm.data.constants import IMAGENET_DEFAULT_MEAN, IMAGENET_DEFAULT_STD
# from torch.nn import Linear

# model = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_224', pretrained=True)
# model.head = Linear(in_features=768, out_features=N_CLASSES, bias=True)
# model.load_state_dict(torch.load("/content/drive/MyDrive/4-1_Capstone/models/D_2.pth"))

# model = model.cuda().eval()

In [23]:
import numpy as np
def visualize_gan(disease_label, epoch, netG = 'unet_128', exception=False):
    '''
    exception : 파, 양배추 = True
    '''
    epoch = str(epoch)
    import shutil

    if os.path.exists("/content/dataset"):
        os.system("rm -rf /content/dataset")

    os.mkdir("/content/dataset")
    for p in ['trainA', 'trainB', 'testA', 'testB']:
        os.mkdir(f"/content/dataset/{p}")

    # train_B 에는 최소 1개의 이미지는 있어야함
    shutil.copy("/content/LeafGAN/media/partial_leaf.png", \
                "/content/dataset/trainB/0.jpg")
    
    images = capstone_images + aihub_images

    if exception == True:
        healthy_images = glob("/content/tmp/images/*")
    else:
        healthy_images = list(filter(lambda x:x.split('/')[-1].split('_')[0] == disease_healthy_map[disease_label], images))

    for image in healthy_images:
        src = image
        file_name = image.split('/')[-1]
        dst = os.path.join("/content/dataset/trainA", file_name)

        shutil.copy(src, dst)

    # 몇 epoch의 모델을 사용할래?
    import zipfile
    with zipfile.ZipFile(f"/content/drive/MyDrive/4-1_Capstone/GAN{disease_label}/GAN{disease_label}.zip", 'r') as zip_ref:
        zip_ref.extract(f"{epoch}_net_G_A.pth", "/content")

    if not os.path.exists("/content/LeafGAN/checkpoints"):
        os.mkdir("/content/LeafGAN/checkpoints")
    if not os.path.exists(f"/content/LeafGAN/checkpoints/GAN{disease_label}"):
        os.mkdir(f"/content/LeafGAN/checkpoints/GAN{disease_label}")

    shutil.copy(f"/content/{epoch}_net_G_A.pth", f"/content/LeafGAN/checkpoints/GAN{disease_label}/latest_net_G_A.pth")
    shutil.copy(f"/content/{epoch}_net_G_A.pth", f"/content/LeafGAN/checkpoints/GAN{disease_label}/latest_net_G_B.pth")

    os.system(f"python test.py --dataroot /content/dataset --name GAN{disease_label} --model leaf_gan  --phase 'train' --num_test 2000 --netG {netG}")

    import shutil
    src = f"/content/LeafGAN/results/GAN{disease_label}/train_latest/images"
    dst = f"/content/drive/MyDrive/4-1_Capstone/GAN{disease_label}/gen_images{epoch}"

    shutil.make_archive(dst, 'zip', src)
    # gen_images    = natsorted(glob(f"/content/LeafGAN/results/GAN{disease_label}/train_latest/images/*"))
    # healthy_images= natsorted(glob("/content/dataset/trainA/*"))



    # import matplotlib.pyplot as plt
    # from PIL import Image

    # for src, gen in zip(healthy_images, gen_images):
    #     plt.subplot(1, 2, 1)
    #     plt.imshow(Image.open(src))

    #     plt.subplot(1, 2, 2)
    #     plt.imshow(Image.open(gen))
        
    #     plt.show()

    # from torch.utils.data import DataLoader
    # batch_size = 256
    # test_loader = DataLoader(MyDataset(tf, gen_images),
    #                         batch_size=batch_size,
    #                         shuffle=True,
    #                         drop_last=True,
    #                         pin_memory=True,
    #                         num_workers=4)
        
    # preds = []
    # Y = int(disease_label)

    # model.eval()
    # with torch.no_grad():
    #     for X in test_loader:
    #         X = X.cuda()

    #         H = model(X).detach().cpu().numpy()
    #         H *= (np.eye(N_CLASSES)[Y] @ crop_mask)

    #         preds.extend(H.tolist())

    # preds = np.array(preds)
    # print(preds.argmax(1))
    # acc = (np.expand_dims(preds.argmax(1), 0) == int(disease_label)).sum() / len(gen_images)

    # print(acc)

In [26]:
# visualize_gan(disease_label = '4', epoch = 200, netG = 'unet_128') # 

!mkdir /content/tmp
!unzip -qq /content/drive/MyDrive/4-1_Capstone/AI_HubDataset/양배추_정상.zip -d /content/tmp/

visualize_gan(disease_label = '39', epoch = 200, netG = 'unet_256', exception = True)

!rm -rf /content/tmp
!unzip -qq /content/drive/MyDrive/4-1_Capstone/AI_HubDataset/파_정상.zip -d /content/tmp/

visualize_gan(disease_label = '45', epoch = 200, netG = 'unet_256', exception = True) # 
visualize_gan(disease_label = '46', epoch = 200, netG = 'unet_256', exception = True) # 

# visualize_gan(disease_label = '47', epoch = 200, netG = 'unet_128') # 47
# visualize_gan(disease_label = '48', epoch = 200, netG = 'unet_128') # 48

In [ ]:
# from torch.utils.data import DataLoader
# batch_size = 256
# disease_label = '47'

# gen_images = glob(f"/content/LeafGAN/results/GAN{disease_label}/train_latest/images/*")

# test_loader = DataLoader(MyDataset(tf, gen_images),
#                         batch_size=batch_size,
#                         shuffle=True,
#                         drop_last=True,
#                         pin_memory=True,
#                         num_workers=4)
    
# preds = []
# Y = int(disease_label)

# model.eval()
# with torch.no_grad():
#     for X in test_loader:
#         X = X.cuda()

#         H = model(X).detach().cpu().numpy()
#         H *= (np.eye(N_CLASSES)[Y] @ crop_mask)

#         preds.extend(H.tolist())

# preds = np.array(preds)


# # print(preds.argmax(1))
# # acc = (np.expand_dims(preds.argmax(1), 0) == int(disease_label)).sum() / len(gen_images)

# # print(acc)

In [ ]:
preds[6]

array([-0.        , -0.        , -0.        ,  0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
        0.        ,  0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.46191132,  0.90523434, -0.        ,
       -0.        ,  0.        , -0.        ,  0.        , -0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , -0.        , -0.40

In [ ]:
# disease_label

'39'

In [ ]:
gen_images    = natsorted(glob(f"/content/LeafGAN/results/GAN{disease_label}/train_latest/images/*"))
healthy_images= natsorted(glob("/content/dataset/trainA/*"))


import matplotlib.pyplot as plt
from PIL import Image

for src, gen in zip(healthy_images, gen_images):
    plt.subplot(1, 2, 1)
    plt.imshow(Image.open(src))

    plt.subplot(1, 2, 2)
    plt.imshow(Image.open(gen))
    
    plt.show()

Output hidden; open in https://colab.research.google.com to view.